# install packages

In [1]:
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-sdtc5ngs
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-sdtc5ngs
  Resolved https://github.com/openai/whisper.git to commit ad3250a846fe7553a25064a2dc593e492dadf040
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 KB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.8 MB/s eta 0:00:00
  Created wheel for open

# mount drive and import packages

In [2]:
!nvidia-smi

Sat Mar 11 00:41:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import os
import datetime
import glob
import pandas as pd
import whisper

path = "/content/drive/MyDrive/uraradi_transcript/"

# identify existing mp3/transcript

In [5]:
mp3_list = []
for sounds_file in glob.glob(path+"sounds/*.mp3"):
    filename = sounds_file.split("/")[-1].split(".")[0]
    mp3_list.append(filename)

csv_list = []
for transcripts_file in glob.glob(path+"transcripts/*.csv"):
    filename = transcripts_file.split("/")[-1].split(".")[0]
    csv_list.append(filename)

# identify target radio

In [9]:
playlist = pd.read_csv(path + "playlist_裏ラジオウルナイト.csv")
playlist["flag"] = playlist["date"].apply(lambda date: (date in mp3_list)&(date not in csv_list))
target_radios = playlist[playlist["flag"]==True][["url", "date"]].reset_index(drop=True).copy()
sounds_not_created = target_radios["date"].reset_index(drop=True).copy()
sounds_not_created

0    2023-03-10
Name: date, dtype: object

# to transcript

In [7]:
model = whisper.load_model("large")

100%|██████████████████████████████████████| 2.87G/2.87G [00:30<00:00, 101MiB/s]


In [10]:
for sound in sounds_not_created:
    print(datetime.datetime.now())
    print(sound)
    mp3 = path + "sounds/" + sound + ".mp3"
    print(mp3)
    result = model.transcribe(mp3, language="ja", verbose=True)
    # result = model.transcribe(mp3, language="ja", verbose=False)

    transcript_column = ["start_s", "end_s", "text"]
    transcript_list = []
    for segment in result["segments"]:
        start_s = int(segment["start"])
        end_s = int(segment["end"])
        row = [start_s, end_s, segment["text"]]
        transcript_list.append(row)

    transcript = pd.DataFrame(transcript_list, columns=transcript_column)

    transcript.to_csv(path + "transcripts/" + sound + ".csv", index=False)

    os.remove(mp3)

2023-03-11 00:44:05.210444
2023-03-10
/content/drive/MyDrive/uraradi_transcript/sounds/2023-03-10.mp3
[00:00.000 --> 00:02.000] 裏ラジオールナイト
[00:12.000 --> 00:16.400] 皆さんこんばんは 日付が変わりまして3月11日土曜日
[00:17.000 --> 00:18.480] 深夜1時
[00:18.480 --> 00:20.480] めちゃくちゃ土曜日なんですけれども
[00:20.760 --> 00:24.680] 金曜深夜のね定期ラジオと言い張っております裏ラジオオールナイト
[00:25.600 --> 00:28.960] 本日もお相手はユーカンキッスアニマルの大浦良加子です
[00:29.520 --> 00:33.160] これテイク3です めちゃくちゃ噛みましたのでね
[00:34.160 --> 00:39.760] テイク1 テイク2 めちゃくちゃ噛みましたので今テイク3を撮っております
[00:41.440 --> 00:44.600] 今回ですねちょっと私のPCの
[00:45.560 --> 00:47.560] 調子がすこぶる悪くてですね
[00:48.080 --> 00:53.560] 動画でお送りしているかと思います 本来はいつも生配信で
[00:54.120 --> 00:56.120] お送りしているんですけれども
[00:56.640 --> 00:58.040] 動画でね
[00:58.040 --> 01:00.040] 一人ぼっちの部屋で
[01:00.040 --> 01:02.040] 一人で喋っています
[01:02.040 --> 01:04.040] 別にいつも変わらないんですけど
[01:04.040 --> 01:06.440] いつも一人ぼっちの部屋で一人で喋ってるんですけど
[01:07.160 --> 01:09.160] やっぱりリアルタイムのレスポンスがね
[01:09.360 --> 01:11.280] まあその
[01:11.280 --> 01:13.280] やっぱりそのコメントとか
[01:13.640 --> 01:18.52